# JUNIOR PERFORMANCE MARKETING ANALYST


## Idealo Case Study - Marketing Data*
<i>*For the purpose of this presentation, the code will be displayed in a detailed and analytical way. One action per step.</i>

<br></br>

### Presented by Kiril Yunakov

### Table of contents:
1. Explore the data
    1. Install library and load the file
    2. Show the count and the different values in each column
2. [Initial findings](#initial_findings)
3. Data wrangling
    1. Format the coloumn names for consistency
    2. Format the datetime of the respective day of the week
    3. Devide the dataset into two sub-datasets - one for Leadouts and one for Earnings
    4. Merge the two datasets into one (left-join) and remove the unnecessary columns
    5. Create a new column for Earnings per Leadout
4. Data Analysis
    1. General Data Analysis
    2. Country Specific Data Analysis


<br></br>

## 1. Explore the data
Firstly, let's explore the dataset and find out what data is available and how it is organized. 

### 1.1. Install library and load the file

In [1]:
! pip install pandas
import pandas as pd
import numpy as np

from IPython.display import IFrame

marketing_data = pd.read_csv(r"C:\Users\User\Downloads\new_data.csv", sep=";") #the separator of the file is a semicolon
print("The libraries have been installed and the file has been loaded.")

The libraries have been installed and the file has been loaded.


In [2]:
#show the first 10 rows of table (dataframe)
marketing_data[:10]

,Day of Transaction Date,Device,Landing Pagetype,Measure Names,MktChannel,Month of Transaction Date,Shop Type,Site,Campaign,source,Year of Transaction Date,Measure values
0,10,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,6
1,10,pc,GoToShop,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,0
2,21,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6
3,21,pc,GoToShop,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,0
4,7,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6
5,7,pc,GoToShop,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,0
6,3,pc,Home,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,6
7,3,pc,Home,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,0
8,5,pc,Home,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,frau,orf,2018,6
9,5,pc,Home,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,frau,orf,2018,0


### 1.2. Show the count and the different values of each column

In [3]:
# count all unique values and add them to a list
device = marketing_data['Device'].unique().tolist()
site = marketing_data['Site'].unique().tolist()
source = marketing_data['source'].unique().tolist()
MktChannel = marketing_data['MktChannel'].unique().tolist()
landing_pagetype = marketing_data['Landing Pagetype'].unique().tolist()
shop_type = marketing_data['Shop Type'].unique().tolist()
month = marketing_data['Month of Transaction Date'].unique().tolist()
year = marketing_data['Year of Transaction Date'].unique().tolist()
campaign = marketing_data['Campaign'].unique().tolist()

#print the count and the values
print("The dataset consists of: \n"
    +(str(len(device)))+" the different Devices: "+str(device)+"\n"
    +str(len(site))+" different Sites: "+str(site)+"\n"
    +str(len(source))+" different Sources: "+str(source)+"\n"
    +str(len(MktChannel))+" different Marketing Channels: "+str(MktChannel)+" \n"
    +str(len(landing_pagetype))+" different Landing Page Types: "+str(landing_pagetype)+" \n"
    +str(len(shop_type))+" different Shop Types: "+str(shop_type)+" \n"
    +str(len(month))+" different Month: "+str(month)+" \n"
    +str(len(year))+" different Year: "+str(year)+" \n"
    +str(len(campaign))+" different Campaigns: "+str(campaign[:10])+"... \n")

The dataset consists of: 
3 the different Devices: ['pc', 'phone', 'tablet']
3 different Sites: ['idealo.at', 'idealo.de', 'idealo.es']
16 different Sources: ['facebook', 'google', 'orf', nan, 'iadserver', 'instagram', 'criteo', 'koop_as', 'bing', 'yandex', 'idealo', 'Bing', 'geizhalsat', 'web', 'shoppydoo', 'Encuentraprecios']
6 different Marketing Channels: ['Display', 'Remarketing', 'SEM:Brand', 'SEM:Generic', 'SEO:Brand', 'SEO:Generic'] 
9 different Landing Page Types: ['GoToShop', 'Home', 'OOP', nan, 'Other', 'PCat', 'Search', 'List', 'Sitemap'] 
4 different Shop Types: ['AMZ_MARKETPLACE', 'AMZ_RETAIL', 'EBY', 'SALESPIXEL'] 
1 different Month: ['October'] 
1 different Year: [2018] 
228 different Campaigns: ['Engagement', 'Engagement_OOP', 'frau', 'DE_PV_DPA_engagement', nan, 'ENGA_RE_desktop', 'dynamicm_doi_b', 'iphonedyn', 'tv-collection', 'Prospecting_New_Users']... 



## 2. Initial findings <a id='initial_findings'></a>
The performance marketing data set includes information about the Idealo campaigns, consisting of `Device`, `Marketing Channel`, `Shop type`, `Site`, `Campaing`, `Source`, `Year of Transaction Date`, `Month of Transaction Date` and `Day of Transaction Date`. The timeframe of the dataset is only for one full month - `October 2018`. The countries of scope are `Austria`, `Germany` and `Spain` via 3 different devices `pc`, `phone`, `tablet`.


The performance of each campaign is measured by two KPIs which can be found under the same column `Measure names`:
* Earnings
* Leadouts
<br>






This makes it impossible to directly estimate the <b>Earnings per Leadouts</b> for each campaign due to the lack of direct connection/relationship between `Measure Names` (Earnings/Leadouts) and `Measure values`. This is caused by the <b>missing unique record id (key)</b> for each campaign.

## 3. Data wrangling
The `Measure values` and the `Measure Names` are in a one-to-one relationship. This means each row represents one measurement and all of the other attributes:
* `Device`
* `Marketing Channel`
* `Shop type`
* `Site`
* `Campaing`
* `source`
* `Year of Transaction Date`
* `Month of Transaction Date`
* `Day of Transaction Date`

for one campaign are identical. Only the KPIs are different. In order to solve this problem, I need to wrangle (manipulate) the dataset first by:
1. Formatting the coloumn names for consistency
2. Formating the datetime with the respective day of the week
3. Deviding the dataset into two sub-datasets - one for __Leadouts__ and one for __Earnings__.
4. Merging the two datasets and remove the unnecessary columns
5. Creating a new column for __Earnings per Leadout__

### 3.1. Format the column names for consistency

In [4]:
marketing_data.columns = marketing_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
marketing_data.head()

,day_of_transaction_date,device,landing_pagetype,measure_names,mktchannel,month_of_transaction_date,shop_type,site,campaign,source,year_of_transaction_date,measure_values
0,10,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,6
1,10,pc,GoToShop,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,0
2,21,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6
3,21,pc,GoToShop,Earnings,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,0
4,7,pc,GoToShop,Leadouts,Display,October,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6


### 3.2. Format the datetime of the respective day of the week

In [5]:
#map the month of the year into integers
m = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 
     'November':11, 'December':12,}
marketing_data.month_of_transaction_date = marketing_data.month_of_transaction_date.map(m)


#combine the date information - day, month, year into a DateTime
marketing_data['date'] = pd.to_datetime(marketing_data.year_of_transaction_date*10000+marketing_data.month_of_transaction_date*100+marketing_data.day_of_transaction_date,format='%Y%m%d')
marketing_data['weekday'] = marketing_data['date'].dt.day_name()

marketing_data.head()

,day_of_transaction_date,device,landing_pagetype,measure_names,mktchannel,month_of_transaction_date,shop_type,site,campaign,source,year_of_transaction_date,measure_values,date,weekday
0,10,pc,GoToShop,Leadouts,Display,10,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,6,2018-10-10,Wednesday
1,10,pc,GoToShop,Earnings,Display,10,AMZ_MARKETPLACE,idealo.at,Engagement,facebook,2018,0,2018-10-10,Wednesday
2,21,pc,GoToShop,Leadouts,Display,10,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6,2018-10-21,Sunday
3,21,pc,GoToShop,Earnings,Display,10,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,0,2018-10-21,Sunday
4,7,pc,GoToShop,Leadouts,Display,10,AMZ_MARKETPLACE,idealo.at,Engagement_OOP,google,2018,6,2018-10-07,Sunday


### 3.3. Devide the dataset into two sub-datasets - one for Leadouts and one for Earnings

In [6]:
# create a new dataframe only from the Leadouts
marketing_data_leadouts = marketing_data[marketing_data.measure_names == "Leadouts"]

# rename the existing column of measure_values into leadouts
marketing_data_leadouts = marketing_data_leadouts.rename(columns={"measure_values": "leadouts"})

In [7]:
# create a new dataframe only from the Earnings
marketing_data_earnings = marketing_data[marketing_data.measure_names == "Earnings"]

# rename the existing column of measure_values into earnings
marketing_data_earnings = marketing_data_earnings.rename(columns={"measure_values": "earnings"})

### 3.4.  Merge the two datasets into one (left-join) and remove the unnecessary columns

In [8]:
# Merge the datasets
result = pd.merge(marketing_data_leadouts, marketing_data_earnings, on=['day_of_transaction_date', 'month_of_transaction_date',
                                                                        'year_of_transaction_date','date','weekday','device', 
                                                                        'landing_pagetype', 'mktchannel', 'shop_type', 'site', 
                                                                        'campaign', 'campaign', 'source'], how='left')

# Delete the unnecessary columns
columns = ['measure_names_x', 'measure_names_y', 'month_of_transaction_date', 'year_of_transaction_date']
result.drop(columns, axis=1, inplace = True) 

# Change the order of the coloumn for better visualization
result = result[['date','day_of_transaction_date','weekday','site','mktchannel','source','campaign','landing_pagetype','device',
                 'shop_type','leadouts','earnings']]

result.head()

,date,day_of_transaction_date,weekday,site,mktchannel,source,campaign,landing_pagetype,device,shop_type,leadouts,earnings
0,2018-10-10,10,Wednesday,idealo.at,Display,facebook,Engagement,GoToShop,pc,AMZ_MARKETPLACE,6,0
1,2018-10-21,21,Sunday,idealo.at,Display,google,Engagement_OOP,GoToShop,pc,AMZ_MARKETPLACE,6,0
2,2018-10-07,7,Sunday,idealo.at,Display,google,Engagement_OOP,GoToShop,pc,AMZ_MARKETPLACE,6,0
3,2018-10-03,3,Wednesday,idealo.at,Display,facebook,Engagement,Home,pc,AMZ_MARKETPLACE,6,0
4,2018-10-05,5,Friday,idealo.at,Display,orf,frau,Home,pc,AMZ_MARKETPLACE,6,0


## 4. Data Analysis

The data analysis for this case will be done by Google Spreadsheets. The following analysis have been created for each country:
* Weekday Analysis
* Source Analysis 
* Channel Analysis
* Device Analysis
* LandingPage Analysis
* Device vs. Source

For the calculations, I used mostly pivot tables with calculated fields. The findings of each analysis can be found in the spreadsheets itself.

### 4.1. General Data Analysis

Market Analysis - https://docs.google.com/spreadsheets/d/1g2Fvg0EaxcibyzZ5c3ppMxjhpWtQhUj_MNgIMCH3tUY/edit?usp=sharing

### 4.2. Country Specific Data Analysis

* Austria - https://docs.google.com/spreadsheets/d/1IpFMNRVotaRiWjSQh1fBS1pUcYgg3jZiqTFNzIsfinM/edit?usp=sharing
* Germany - https://docs.google.com/spreadsheets/d/13qNTfrU4MK4ktUJmA2lEQ4vSNSPpBzan_Qx8HoB5PL0/edit?usp=sharing
* Spain - https://docs.google.com/spreadsheets/d/1ErqwmZVr4qKEbImL1StidxdYYH_5nmwhJyrxteVy44U/edit?usp=sharing

Thank you for your attention!

Best regards, <br>
Kiril Yunakov